In [1]:
# depenencies
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

from sklearn.model_selection import cross_val_score
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.compose import TransformedTargetRegressor
from sklearn.model_selection import train_test_split
from sklearn.linear_model import Lasso
from sklearn.kernel_ridge import KernelRidge
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import KFold,StratifiedKFold, RepeatedKFold
from sklearn.decomposition import PCA,KernelPCA
from sklearn.model_selection import cross_validate


preapre_x_y will will make X and y vector.

\* if **need_filter** is equals to True we will remove each column with low correlation to 'time_to_eruption' (smaller then a givn threshold)

In [2]:
def prepare_x_y(data,need_filter=False,thresh=0.01):
    to_drop=['time_to_eruption'] if 'time_to_eruption' in data.columns else []
    if need_filter:
        to_drop+=(list(data.corr()[abs(data.corr()['time_to_eruption'])<thresh].index))
   
    X=data.drop(to_drop,axis=1)
    if 'segment_id' in X.columns:
        X=X.drop(['segment_id'],axis=1)
    Y=data['time_to_eruption']
    return X,Y,to_drop

In [3]:
data_pre_name='train_set_data'
data_pre=pd.read_csv(data_pre_name+'.csv')
# filling nulls with mean value
data_pre=data_pre.fillna(data_pre.mean())
data_pre=data_pre.drop('Unnamed: 0',axis=1)
data_pre.head()

,segment_id,sensor_1_sum,sensor_1_mean,sensor_1_std,sensor_1_var,sensor_1_max,sensor_1_min,sensor_1_skew,sensor_1_mad,sensor_1_kurtosis,...,missed_percent_sensor1,missed_percent_sensor2,missed_percent_sensor3,missed_percent_sensor4,missed_percent_sensor5,missed_percent_sensor6,missed_percent_sensor7,missed_percent_sensor8,missed_percent_sensor9,missed_percent_sensor10
0,288431696,-71338.0,-1.188947,263.712449,69544.255815,1761.0,-1878.0,-0.011270,199.183223,2.666417,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.000000,0.0
1,640300977,-278979.0,-4.649573,307.786143,94732.309965,1351.0,-1341.0,-0.054428,240.888460,0.376833,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.085815,0.0
2,1488245608,262769.0,4.379410,381.276530,145371.792095,1422.0,-1446.0,0.030110,306.953591,-0.180576,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0
3,1075300188,-310290.0,-5.171414,301.818525,91094.421933,1060.0,-1528.0,-0.081617,239.896117,0.060633,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0
4,372844561,129520.0,2.158631,696.277869,484802.871536,2903.0,-3451.0,0.020535,551.750742,0.114210,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0


In [4]:
test_pre_name='test_set_data'
test_pre=pd.read_csv(test_pre_name+'.csv')
test_pre=test_pre.drop('Unnamed: 0',axis=1)
test_pre.head()

,segment_id,sensor_1_sum,sensor_1_mean,sensor_1_std,sensor_1_var,sensor_1_max,sensor_1_min,sensor_1_skew,sensor_1_mad,sensor_1_kurtosis,...,missed_percent_sensor1,missed_percent_sensor2,missed_percent_sensor3,missed_percent_sensor4,missed_percent_sensor5,missed_percent_sensor6,missed_percent_sensor7,missed_percent_sensor8,missed_percent_sensor9,missed_percent_sensor10
0,1287137774,-2847639.0,-47.459859,6287.375712,3.953109e+07,23147.0,-25732.0,-0.044194,5023.409358,-0.069530,...,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.021633,0.0
1,1989256535,-206217.0,-3.436893,379.397354,1.439424e+05,1931.0,-1840.0,-0.020218,294.097312,0.895616,...,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.018416,0.0
2,343649592,-6034.0,-0.100565,982.125574,9.645706e+05,10481.0,-10592.0,-0.118749,665.956863,8.537645,...,0.0,1.0,0.0,0.0,0.975767,0.0,1.0,0.0,0.000000,0.0
3,1392908532,-377376.0,-6.289495,2091.236887,4.373272e+06,8539.0,-12196.0,-0.042707,1659.461261,0.302375,...,0.0,0.0,0.0,0.0,0.985534,0.0,0.0,0.0,0.000000,0.0
4,1296158394,29168.0,0.486125,521.973942,2.724568e+05,2267.0,-2427.0,-0.017905,409.219533,0.389365,...,0.0,0.0,0.0,0.0,1.000000,0.0,0.0,0.0,0.010066,0.0


In [5]:
X,Y,to_drop=prepare_x_y(data_pre,True,0.01)

### Using Lasso to get the important features

In [6]:
# def get_importat_features_by_lasso():
rkf = RepeatedKFold(n_splits=5, n_repeats=4, random_state=1)
scaler_lasso = StandardScaler()
lasso = Lasso(alpha=2.5, fit_intercept=True, normalize=False, max_iter=20_000, tol=1e-3, positive=False, random_state=1, selection='random')
tt_lasso = TransformedTargetRegressor(regressor=lasso, func=np.sqrt, inverse_func=np.square)
pipe_lasso = Pipeline([('scaler', scaler_lasso), ('reg', tt_lasso)])
cv_lasso = cross_val_score(pipe_lasso, 
                           X, 
                           Y, 
                           scoring='neg_mean_absolute_error', 
                           cv=rkf, 
                           n_jobs=4, 
                           verbose=True, 
                           error_score='raise')
print(f'Mean MAE is {(cv_lasso*-1).mean()}')
pipe_lasso.fit(X, Y)
lasso_coefs = pipe_lasso.named_steps['reg'].regressor_.coef_
important_cols = X.columns[lasso_coefs != 0]


[Parallel(n_jobs=4)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done  20 out of  20 | elapsed:   22.9s finished


Mean MAE is 9783612.603157079


In [7]:
def get_kernal_pca(X,kernel='linear',n_components=0.999,optionals={'gamma':None}):
    kpca=KernelPCA(kernel=kernel,gamma=optionals['gamma'],remove_zero_eig=True)
    kpca.fit(X)
    ratio=kpca.lambdas_.cumsum()/kpca.lambdas_.sum()
    n_comp=(ratio < n_components).sum()
    
    return KernelPCA(kernel=kernel,n_components=n_comp,gamma=optionals['gamma'],remove_zero_eig=True)

In [8]:
from sklearn.model_selection import GridSearchCV

from sklearn.model_selection import cross_validate
X=data_pre[important_cols]
Y=data_pre['time_to_eruption']


# model=KernelRidge(kernel ='rbf',alpha=1.0)
model = GridSearchCV(KernelRidge(kernel='rbf', gamma=0.1),
                  param_grid={"alpha": [1e0, 0.1, 1e-2],
                              "gamma": np.logspace(-2, 2, 3)})


x_train,x_test,y_train,y_test=train_test_split(X[important_cols],Y,test_size=0.2)

scaler1 = StandardScaler()
# Fit on training set only.
scaler1.fit(x_train)
# Apply transform to both the training set and the test set.
train_img = scaler1.transform(x_train)
test_img = scaler1.transform(x_test)
# pca=KernelPCA(kernel='linear',remove_zero_eig=True)
pca = get_kernal_pca(train_img)

model = GridSearchCV(KernelRidge(kernel='rbf', gamma=0.1),
                  param_grid={"alpha": [1e0, 0.1, 1e-2, 1e-3],
                              "gamma": np.logspace(-2, 2, 3)})

pca.fit(train_img)
train_pca = pca.transform(train_img)
test_pca = pca.transform(test_img)

naive_model = cross_validate(model, train_pca ,y_train, cv=5,
                                  return_estimator=True,return_train_score=True,
                                  scoring=['neg_mean_absolute_error'])
idx_best_estimator = np.argmax(naive_model['test_neg_mean_absolute_error'])
predictions = naive_model['estimator'][idx_best_estimator].predict(test_pca)

mae = mean_absolute_error(np.array(y_test),np.array(predictions))

print("Train MAE: %.3f" %-naive_model['train_neg_mean_absolute_error'][idx_best_estimator])
#print("Train RMSE: %.3f" %-naive_model['train_neg_root_mean_squared_error'][word_best_estimator])
print("Validation MAE: %.3f" %-naive_model['test_neg_mean_absolute_error'][idx_best_estimator])
#print("Validation RMSE: %.3f" %-naive_model['test_neg_root_mean_squared_error'][word_best_estimator])
print("Test MAE: %.3f" %mae)

Train MAE: 4789536.059
Validation MAE: 7027258.021
Test MAE: 7513450.150


In [9]:
to_drop.remove('time_to_eruption')

In [10]:
test_pre_new=test_pre.drop(to_drop,axis=1)
real_test=  scaler1.transform(test_pre_new[important_cols])
real_test = pca.transform(real_test)
# real_test = scaler2.transform(real_test)

In [11]:
prediction_real=naive_model['estimator'][idx_best_estimator].predict(real_test)

In [12]:
sub = pd.read_csv("sample_submission.csv")
sub['time_to_eruption'] = prediction_real
sub.head()

,segment_id,time_to_eruption
0,1000213997,1.571189e+05
1,100023368,9.424082e+06
2,1000488999,4.339253e+07
3,1001028887,-1.539703e+07
4,1001857862,2.913394e+07
